In [69]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are tasked with generating fusion and inversion functions for arithmetic formulas based on the Semantic Fusion methodology. Your response must strictly adhere to the following steps and output format.
---
**Background:**
Semantic Fusion is a process to combine two formulas \( \phi_1 \) and \( \phi_2 \) into a new fused formula \( \phi_12 \) . This involves:

1. **Fusion Function** \( f(x, y) \): Combines variables \( x \) and \( y \) into a new variable \( z \).
Fusion functions must combine variables into a fused variable \( z \), while maintaining logical equivalence with the input formulas. 

2. **Inversion Functions**:
   - \( r_x(y, z) \): Recovers \( x \) from \( y \) and \( z \).
   - \( r_y(x, z) \): Recovers \( y \) from \( x \) and \( z \).
   Inversion functions must accurately reconstruct the original variables \( x \) and \( y \) from \( z \).

   3. **Fused Formula**: Rewrites the constraints in \( \phi_1 \) and \( \phi_2 \) in terms of \( z \), ensuring that the constraints are equisatisfiable with the originals.

Refer to the following table for predefined fusion and inversion functions categorized by types (Int, Real, String):

#### **Table: Fusion and Inversion Functions**
| Type   | Fusion Function \( f(x, y) \)    | \( r_x(y, z) \)                          | \( r_y(x, z) \)                            |
|--------|----------------------------------|------------------------------------------|--------------------------------------------|
| **Int**  | \( x + y \)                      | \( z - y \)                              | \( z - x \)                                |
|        | \( x + c + y \)                  | \( z - c - y \)                          | \( z - c - x \)                            |
|        | \( x * y \)                 | \( z \, div \, y \)                      | \( z \, div \, x \)                        |
|        | \( c_1 * x + c_2 * y + c_3 \) | \( (z - c_2 * y - c_3) \, div \, c_1 \) | \( (z - c_1 * x - c_3) \, div \, c_2 \) |
| **Real** | \( x + y \)                      | \( z - y \)                              | \( z - x \)                                |
|        | \( x + c + y \)                  | \( z - c - y \)                          | \( z - c - x \)                            |
|        | \( x * y \)                 | \( z / y \)                              | \( z / x \)                                |
|        | \( c_1 * x + c_2 * y + c_3 \) | \( (z - c_2 * y - c_3) / c_1 \)      | \( (z - c_1 * x - c_3) / c_2 \)       |
| **String** | \( x \, str++ \, y \)           | \( str.substr(z, 0, str.len(x)) \)       | \( str.substr(z, str.len(x), str.len(y)) \)|
|        | \( x \, str++ \, c \, str++ \, y \) | \( str.substr(z, 0, str.len(x)) \)       | \( str.replace(str.replace(z, x, ""), c, "") \)|


### SMT-LIB Syntax Instructions:
1.	Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
2.	Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
3.	Assertions: Use assert to specify constraints: (assert <constraint>).
4.	Boolean Expressions:
	•	Logical AND: (and <expr1> <expr2> ...)
	•	Logical OR: (or <expr1> <expr2> ...)
	•	Logical NOT: (not <expr>)
5.	Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
6.	Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
7.	Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
8.	Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
9.	Quantifiers: Use forall and exists for quantified expressions:
	•	Universal: (forall ((x <type>) ...) <body>)
	•	Existential: (exists ((x <type>) ...) <body>)
10.	Constants: Declare constants directly using their types:
	•	Integers: 1, -3
	•	Real numbers: 1.0, -2.5
	•	Booleans: true, false
11.	Parentheses Matching: Ensure all expressions are enclosed in properly matched parentheses.
12.	Comments: Use ; for single-line comments in SMT-LIB.
13. Avoid invalid constructs such as `f(x y) =` or `verify`. SMT-LIB does not support such constructs.

---

**Output Format:**
Your response must strictly follow this format:

1. **Fusion Function**:
   ```smt
   (define-fun f ((x Real) (y Real)) Real
     [insert fusion function here])

2. **Inversion Functions**:
    (define-fun r_x ((y Real) (z Real)) Real
    [insert inversion function for x])

    (define-fun r_y ((x Real) (z Real)) Real
    [insert inversion function for y])

3. **Fused Formula (SMT-LIB)**:
    (declare-fun z () Real)
    (assert (and [insert fused formula here]))

4.	**Validation**:
    •	Example Inputs: ( x = 2, y = -1 )
    •	Computed ( z = f(x, y) ): [insert result here].
    •	Reconstructed ( x ): [insert result here].
    •	Reconstructed ( y ): [insert result here].
    •	Verify that ( \phi_1 ) and ( \phi_2 ) hold with the reconstructed values.
  
**Notes:**
  1.	Use ( z ) as the fused variable.
  2.	Ensure the fused formula preserves equisatisfiability.
  3.    The fused formula must be valid and consistent with the fusion function.
  3.	Provide valid SMT-LIB syntax only.
  4.	Avoid replicating examples; generate outputs specific to the provided input formulas.

        """
        ),
        (
            "human",
            """
**Input Formulas:**
- \( \phi_1: {formula1} \)
- \( \phi_2: {formula2} \)
"""
    ),
]
)

In [70]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [71]:
chain = prompt | llm
formula1 = "(declare-fun x () Int)\n(declare-fun w () Bool)\n(assert (= x (- 1)))\n(assert (= w (= x (- 1))))\n(assert w)"
formula2 = "(declare-fun y () Int)\n(declare-fun v () Bool)\n(assert (= v (not (= y (- 1)))))\n(assert (ite v false (= y (- 1))))"

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

**Fusion Function for \( \phi_1 \):**
```smt
(define-fun f ((x Real) (w Real)) Real
  (+ x w))
```
This fusion function combines the variables `x` and `w` into a new variable `z`, where `z = x + w`.

**Inversion Functions for \( \phi_1 \):**
```smt
(define-fun r_x ((y Real) (z Real)) Real
  (- z y))

(define-fun r_w ((x Real) (z Real)) Real
  z)
```
These inversion functions recover the original variables `x` and `w` from the fused variable `z`. The function `r_x` recovers `x` by subtracting `y` from `z`, while `r_w` recovers `w` directly from `z`.

**Fused Formula for \( \phi_1 \):**
```smt
(declare-fun z () Real)
(assert (and (= z (- 1 + w))))
(assert (= w (= z - r_x(z))))
```
This fused formula rewrites the constraints in `\( \phi_1 )` in terms of the fused variable `z`, ensuring that the constraints are equisatisfiable with the originals.

**Validation for \( \phi_1 \):**
```smt
(verify (assert (= z (- 1 + w))))
(assert (= w (= z - r_x(z))))
(assert (= x (- 1)))
(assert (= w (= x (